In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Load content
X, y = [], []
with open('/home/roger/Desktop/rmi/data.txt') as fin:
    for line in fin:
        arr = line.strip().split(',')
        v1 = float(arr[0])
        v2 = float(arr[1])
        v = int(arr[2])
        X.append([v1, v2])
        y.append(v)
        
X = np.array(X)
y = np.array(y)
y = y.reshape((-1, 1)) 
print X.shape
print y.shape

In [ ]:
# Get positive and negative vectors
approved = []
rejected = []
for arr, lbl in zip(X, y):
    x1, x2 = arr
    if lbl == 1:
        approved.append(arr)
    else:
        rejected.append(arr)
approved = np.array(approved)
rejected = np.array(rejected)

In [ ]:
# Show how data is distributed
fig = plt.figure(figsize=(8,8))
plt.plot(approved[:,0], approved[:,1], 'rx', label='approved')
plt.plot(rejected[:,0], rejected[:,1], 'bo', label='rejected')
plt.legend(loc='upper right')
plt.xlabel('Exam 1 score')
plt.ylabel('Exam 2 score')
plt.show()

## Implement the Sigmoid function

The sigmoid function is defined as:

$$g(z) = \frac{1}{1 + e^{-z}}$$

In [ ]:
## Implement the Sigmoid function
def sigmoid_for(z):
    """ Sigmoid function using for """
    g = np.zeros(z.shape)
    w, h = z.shape
    for i in range(0, w):
        for j in range(0, h):
            g[i, j] = 1./(1 + np.exp(-z[i,j]))
    return g


def sigmoid(z):
    """ Sigmoid function """
    return 1 / (1 + np.exp(-z))

In [ ]:
# Test the Sigmoid function

range_vals = np.arange(-10, 10, step=1)
zero_vals = np.array(20*[0.5])
fig, ax = plt.subplots(figsize=(12,8))
ax.plot(range_vals, sigmoid(range_vals), 'r')
ax.plot(range_vals, zero_vals, '#0F0F0F')
plt.show()

# Cost Function

The cost function as a logistic regression hypothesis is defined as:

$$J(\theta) = \frac{1}{m} \sum_{i=1}^{m} [ -y^{(i)}\ log (h_{\theta}(x^{(i)})) - (1 - y^{(i)})\ log(1 - h_{\theta}(x^{(i)})) ]$$

where 
$$h_{\theta}(x^{(i)}) = \sigma(X\theta^T) = g(X\theta^T)$$

In [ ]:
# Implementation of the Cost Function
def cost_function(theta, X, y):
    """ Compute the cost function of theta """
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    J = np.sum(first - second) / (len(X))
    return J

In [ ]:
# Add column to X since it makes computation easier
m, n = X.shape
X = np.c_[np.ones(m), X]
theta = np.zeros(n+1)
J = cost_function(theta, X, y)
print J

## Gradient

The gradient of the cost is a vector of the same length as $\theta$ where the $j$th element (for $j=0, 1,..., n$) is defined as follows:

$$\frac{\partial J(\theta)}{\partial \theta_j} = \frac{1}{m} \sum_{i-1}^{m} (h_{\theta}(x^{(i)}) - y^{(i)})x_j^{(i)}$$

In [ ]:
def gradient_descent(theta, X, y):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)
    
    error = sigmoid(X * theta.T) - y
    
    for i in range(parameters):
        term = np.multiply(error, X[:,i])
        grad[i] = np.sum(term) / len(X)
    
    return grad

print gradient(theta, X, y)

In [ ]:
import scipy.optimize as opt
result = opt.fmin_tnc(func=cost_function, x0=theta, fprime=gradient_descent, args=(X, y))
cost_function(result[0], X, y)

In [ ]:
def predict(theta, X):
    probability = sigmoid(X * theta.T)
    return [1 if x >= 0.5 else 0 for x in probability]

theta_min = np.matrix(result[0])
predictions = predict(theta_min, X)
scores = []
for a, b in zip(predictions, y):
    if (a and b) or (not a and not b):
        scores.append(1)
    else:
        scores.append(0)
accuracy = (sum(map(int, scores)) % len(scores))
print 'accuracy = {0}%'.format(accuracy)